In [468]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar

#read the users table from a csv file
df = pd.read_csv('CohertAnalysis_user.csv')
df['date_joined'] = pd.to_datetime(df['date_joined'])
#extract all users registered in 2014
dfUsers2014 = df[df['date_joined'].dt.year == 2014]
#Add a new column “CohortMonth”, dtype is numerical value corr to month
dfUsers2014['CohortMonth'] = dfUsers2014['date_joined'].dt.month

#read the orders table
dfOrders = pd.read_csv('CohertAnalysis_orders.csv')
dfOrders['order_date'] = pd.to_datetime(dfOrders['order_date'])
#Add a new column “DaysPastSignOn”, dtype is numerical value corr to difference between order date and date_joined as customer
dfOrders['DaysPastSignOn']=np.zeros(len(dfOrders))
for i in range(0, len(dfOrders)):
    dtUserJoined = (dfUsers2014.query(dfUsers2014['user_id']==dfOrders.user_id.iloc[i])['date_joined']).iloc[0]
    dfOrders['DaysPastSignOn'][i]= abs((dfOrders.iloc[i]['order_date']-dtUserJoined).days)

#left join orders and users tables where orders were placed within one year (365 days) of date_joined
temp = pd.merge(dfOrders[dfOrders['DaysPastSignOn']<365], dfUsers2014, how='left')
#sum all order_amounts for users joined in each month in 2014
dfTemp = temp.groupby(['CohortMonth']).agg({'user_id':  pd.Series.nunique, 'order_amount': 'sum'}).reset_index()

#Create new output table containing data for CohortMonth and Average Cohort Spending 
dfCohorts = pd.DataFrame({' CohortMonth': calendar.month_name[1:13], 'Average Cohort Spending': ['  ']* 12})
for x in range(12):
    temp = dfTemp[['order_amount','user_id']].where(dfTemp['CohortMonth'] == x+1)
    if temp.notnull().values.any():
        dfCohorts['Average Cohort Spending'][x] = (temp['order_amount']/ temp['user_id']).at[0]
dfCohorts


C:\Users\joisi\Anaconda2\lib\site-packages\ipykernel\__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,CohortMonth,Average Cohort Spending
0,January,
1,February,
2,March,
3,April,
4,May,
5,June,
6,July,
7,August,
8,September,221.225
9,October,
